<a href="https://colab.research.google.com/github/bramyeon/sicss-happiness/blob/main/preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b>World Happiness:</b> Data Preprocessing and Cleaning
Summer Institute in Computational Social Science (SICSS-Korea) 2024 Team Project  

<b>Author:</b> Bryan Nathanael Wijaya ([bryannwijaya@kaist.ac.kr](mailto:bryannwijaya@kaist.ac.kr))  
<b>Team Members (ABC order):</b> Inkoo Kang, Ju Hee Jeung, Kyungmin Lee, Yumi Park

## Preliminaries
This notebook is set to run in Google Colaboratory environment. It assumes that the notebook is located at `/content/drive/My Drive/Colab Notebooks/sicss-korea-2024` and the dataset is saved at `data` directory under the previous path, which in turns contains a `clean` subdirectory to save preprocessed datasets. If this is not the case, make sure to modify the `ROOT_DIR` and `DATA_DIR` accordingly, and make a `clean` subdirectory under `DATA_DIR`.

## Datasets
- `data/clean/world-happiness-2005-2023.csv` (1), `data/clean/world-happiness-2024.csv` (2): https://www.kaggle.com/datasets/jainaru/world-happiness-report-2024-yearly-updated/data <br>The world happiness data for 2005-2023 serves as our <b>main</b> dataset, while 2024 data is auxiliary (<b>WARNING:</b> The distribution of the happiness factors in 2024 is different from that in 2005-2023 data, so do NOT merge this to the 2005-2023 data and simply use for separate analysis or as reference only)  
- `data/clean/temperature-2005-2023.csv` (3): https://www.kaggle.com/datasets/subhamjain/temperature-of-all-countries-19952020 <br>This dataset contains the daily temperature of the main cities in each country, which we preprocessed to get the annual average temperature for each country for each (country, year) cases that are considered in our <b>main</b> dataset.
- `data/clean/world-bank-2005-2023.csv` (4): https://databank.worldbank.org/reports.aspx?source=2&series=AG.LND.PRCP.MM&country=# <br>This is an auxiliary dataset taken from the World Bank Dataset to help with our analysis or happiness model training. The elements considered are hand-picked by me (Bryan), where I tried to take those that seem to have some influence on the nation's happiness. (Also since we do not want to have more features than our number of entries, since in that case, model training will likely not converge!) Do let me know if there is any other feature that you would like to additionally consider for our project.
- `data/clean/combined.csv`: This is the combination of datasets (1), (3), and (4). Please use with caution and feel free to remove columns that you think are irrelevant!

### If you find anything suspicious or wrong, do not hesitate to let me know via [email](mailto:bryannwijaya@kaist.ac.kr). Happy coding!


In [ ]:
import os
from google.colab import drive
drive.mount("/content/drive")

ROOT_DIR = '/content/drive/My Drive/Colab Notebooks/sicss-korea-2024'
DATA_DIR = 'data'
DATA_DIR = os.path.join(ROOT_DIR, DATA_DIR)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
for dirname, _, filenames in os.walk(DATA_DIR):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/content/drive/My Drive/Colab Notebooks/sicss-korea-2024/data/world-bank-group.csv
/content/drive/My Drive/Colab Notebooks/sicss-korea-2024/data/city-temperature.csv
/content/drive/My Drive/Colab Notebooks/sicss-korea-2024/data/world-happiness-report-2024.csv
/content/drive/My Drive/Colab Notebooks/sicss-korea-2024/data/world-happiness-report-updated-2024.csv


## World Happiness Data (2005-2023) + 2024 (auxiliary)

1. Adjusted column names for better handling
2. Changed Hong Kong and Taiwan names for better political neutrality
3. Added region category

In [ ]:
df = pd.read_csv(os.path.join(DATA_DIR, "world-happiness-report-updated-2024.csv"), encoding='latin-1')
df.describe()

,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
count,2363.000000,2363.000000,2335.000000,2350.000000,2300.000000,2327.000000,2282.000000,2238.000000,2339.000000,2347.000000
mean,2014.763860,5.483566,9.399671,0.809369,63.401828,0.750282,0.000098,0.743971,0.651882,0.273151
std,5.059436,1.125522,1.152069,0.121212,6.842644,0.139357,0.161388,0.184865,0.106240,0.087131
min,2005.000000,1.281000,5.527000,0.228000,6.720000,0.228000,-0.340000,0.035000,0.179000,0.083000
25%,2011.000000,4.647000,8.506500,0.744000,59.195000,0.661000,-0.112000,0.687000,0.572000,0.209000
50%,2015.000000,5.449000,9.503000,0.834500,65.100000,0.771000,-0.022000,0.798500,0.663000,0.262000
75%,2019.000000,6.323500,10.392500,0.904000,68.552500,0.862000,0.093750,0.867750,0.737000,0.326000
max,2023.000000,8.019000,11.676000,0.987000,74.600000,0.985000,0.700000,0.983000,0.884000,0.705000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2363 entries, 0 to 2362
Data columns (total 11 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Country name                      2363 non-null   object 
 1   year                              2363 non-null   int64  
 2   Life Ladder                       2363 non-null   float64
 3   Log GDP per capita                2335 non-null   float64
 4   Social support                    2350 non-null   float64
 5   Healthy life expectancy at birth  2300 non-null   float64
 6   Freedom to make life choices      2327 non-null   float64
 7   Generosity                        2282 non-null   float64
 8   Perceptions of corruption         2238 non-null   float64
 9   Positive affect                   2339 non-null   float64
 10  Negative affect                   2347 non-null   float64
dtypes: float64(9), int64(1), object(1)
memory usage: 203.2+ KB


In [ ]:
df.columns = [elem.lower().replace('country name', 'country') for elem in df.columns]
df['country'] = [elem.replace(' S.A.R. of China','').replace(' Province of China','') for elem in df['country']]
df.head(5)

,country,year,life ladder,log gdp per capita,social support,healthy life expectancy at birth,freedom to make life choices,generosity,perceptions of corruption,positive affect,negative affect
0,Afghanistan,2008,3.724,7.350,0.451,50.5,0.718,0.164,0.882,0.414,0.258
1,Afghanistan,2009,4.402,7.509,0.552,50.8,0.679,0.187,0.850,0.481,0.237
2,Afghanistan,2010,4.758,7.614,0.539,51.1,0.600,0.118,0.707,0.517,0.275
3,Afghanistan,2011,3.832,7.581,0.521,51.4,0.496,0.160,0.731,0.480,0.267
4,Afghanistan,2012,3.783,7.661,0.521,51.7,0.531,0.234,0.776,0.614,0.268


In [ ]:
df.isnull().sum()

country                               0
year                                  0
life ladder                           0
log gdp per capita                   28
social support                       13
healthy life expectancy at birth     63
freedom to make life choices         36
generosity                           81
perceptions of corruption           125
positive affect                      24
negative affect                      16
dtype: int64

In [ ]:
# Exclude filling non-numeric columns
# numeric_cols = df.select_dtypes(include=np.number).columns
# df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

In [ ]:
df2024 = pd.read_csv(os.path.join(DATA_DIR, "world-happiness-report-2024.csv"), encoding='latin-1')
df2024.describe()

,Ladder score,upperwhisker,lowerwhisker,Log GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Dystopia + residual
count,143.000000,143.000000,143.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000
mean,5.527580,5.641175,5.413972,1.378807,1.134329,0.520886,0.620621,0.146271,0.154121,1.575914
std,1.170717,1.155008,1.187133,0.425098,0.333317,0.164923,0.162492,0.073441,0.126238,0.537459
min,1.721000,1.775000,1.667000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.073000
25%,4.726000,4.845500,4.606000,1.077750,0.921750,0.398000,0.527500,0.091000,0.068750,1.308250
50%,5.785000,5.895000,5.674000,1.431500,1.237500,0.549500,0.641000,0.136500,0.120500,1.644500
75%,6.416000,6.507500,6.319000,1.741500,1.383250,0.648500,0.736000,0.192500,0.193750,1.881750
max,7.741000,7.815000,7.667000,2.141000,1.617000,0.857000,0.863000,0.401000,0.575000,2.998000


In [ ]:
df2024.columns = [elem.lower().replace('country name', 'country').replace('regional indicator', 'region') for elem in df2024.columns]
df2024['country'] = [elem.replace(' S.A.R. of China','').replace(' Province of China','') for elem in df2024['country']]
df2024.head(5)

,country,region,ladder score,upperwhisker,lowerwhisker,log gdp per capita,social support,healthy life expectancy,freedom to make life choices,generosity,perceptions of corruption,dystopia + residual
0,Finland,Western Europe,7.741,7.815,7.667,1.844,1.572,0.695,0.859,0.142,0.546,2.082
1,Denmark,Western Europe,7.583,7.665,7.500,1.908,1.520,0.699,0.823,0.204,0.548,1.881
2,Iceland,Western Europe,7.525,7.618,7.433,1.881,1.617,0.718,0.819,0.258,0.182,2.050
3,Sweden,Western Europe,7.344,7.422,7.267,1.878,1.501,0.724,0.838,0.221,0.524,1.658
4,Israel,Middle East and North Africa,7.341,7.405,7.277,1.803,1.513,0.740,0.641,0.153,0.193,2.298


In [ ]:
df2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143 entries, 0 to 142
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   country                       143 non-null    object 
 1   region                        143 non-null    object 
 2   ladder score                  143 non-null    float64
 3   upperwhisker                  143 non-null    float64
 4   lowerwhisker                  143 non-null    float64
 5   log gdp per capita            140 non-null    float64
 6   social support                140 non-null    float64
 7   healthy life expectancy       140 non-null    float64
 8   freedom to make life choices  140 non-null    float64
 9   generosity                    140 non-null    float64
 10  perceptions of corruption     140 non-null    float64
 11  dystopia + residual           140 non-null    float64
dtypes: float64(10), object(2)
memory usage: 13.5+ KB


In [ ]:
df2024.isnull().sum()

country                         0
region                          0
ladder score                    0
upperwhisker                    0
lowerwhisker                    0
log gdp per capita              3
social support                  3
healthy life expectancy         3
freedom to make life choices    3
generosity                      3
perceptions of corruption       3
dystopia + residual             3
dtype: int64

In [ ]:
# Exclude non-numeric columns to fill missing values
# numeric_cols = df2024.select_dtypes(include=np.number).columns
# df2024[numeric_cols] = df2024[numeric_cols].fillna(df2024[numeric_cols].mean())

In [ ]:
df2024['year'] = 2024
df2024.head(5)

,country,region,ladder score,upperwhisker,lowerwhisker,log gdp per capita,social support,healthy life expectancy,freedom to make life choices,generosity,perceptions of corruption,dystopia + residual,year
0,Finland,Western Europe,7.741,7.815,7.667,1.844,1.572,0.695,0.859,0.142,0.546,2.082,2024
1,Denmark,Western Europe,7.583,7.665,7.500,1.908,1.520,0.699,0.823,0.204,0.548,1.881,2024
2,Iceland,Western Europe,7.525,7.618,7.433,1.881,1.617,0.718,0.819,0.258,0.182,2.050,2024
3,Sweden,Western Europe,7.344,7.422,7.267,1.878,1.501,0.724,0.838,0.221,0.524,1.658,2024
4,Israel,Middle East and North Africa,7.341,7.405,7.277,1.803,1.513,0.740,0.641,0.153,0.193,2.298,2024


In [ ]:
df2024['region'].unique()

array(['Western Europe', 'Middle East and North Africa',
       'North America and ANZ', 'Latin America and Caribbean',
       'Central and Eastern Europe', 'Southeast Asia', 'East Asia',
       'Commonwealth of Independent States', 'Sub-Saharan Africa',
       'South Asia'], dtype=object)

In [ ]:
len(set(df['country'])), len(set(df2024['country']))

(165, 143)

In [ ]:
df_countries = set(df['country'])
df2024_countries = set(df2024['country'])

countries = list(df_countries - df2024_countries)
for country in countries:
    print(country)

Belarus
Belize
Maldives
Turkmenistan
Trinidad and Tobago
Somaliland region
Rwanda
Oman
Angola
Cuba
Suriname
Burundi
South Sudan
Bhutan
Haiti
Somalia
Syria
Sudan
Qatar
Djibouti
Türkiye
Central African Republic
Guyana


In [ ]:
region_dic = {'Cuba': 'Latin America and Caribbean',
              'Suriname': 'Latin America and Caribbean',
              'Burundi': 'Sub-Saharan Africa',
              'Haiti': 'Latin America and Caribbean',
              'Guyana': 'Latin America and Caribbean',
              'Somalia': 'Sub-Saharan Africa',
              'Syria': 'Middle East and North Africa',
              'Belize': 'Latin America and Caribbean',
              'Maldives': 'South Asia',
              'South Sudan': 'Sub-Saharan Africa',
              'Rwanda': 'Sub-Saharan Africa',
              'Qatar': 'Middle East and North Africa',
              'Central African Republic': 'Sub-Saharan Africa',
              'Sudan': 'Middle East and North Africa',
              'Türkiye': 'Middle East and North Africa',
              'Oman': 'Middle East and North Africa',
              'Bhutan': 'South Asia',
              'Belarus': 'Central and Eastern Europe',
              'Trinidad and Tobago': 'Latin America and Caribbean',
              'Angola': 'Sub-Saharan Africa',
              'Somaliland region': 'Sub-Saharan Africa',
              'Turkmenistan': 'Commonwealth of Independent States',
              'Djibouti': 'Sub-Saharan Africa'}

for country, region in zip(df2024['country'], df2024['region']):
    region_dic[country] = region

In [ ]:
df['region'] = [region_dic[elem] for elem in df['country']]

In [ ]:
df = df[['country', 'year', 'region', 'life ladder', 'log gdp per capita', 'social support', 'healthy life expectancy at birth', 'freedom to make life choices', 'generosity', 'perceptions of corruption', 'positive affect', 'negative affect']]
df2024 = df2024[['country', 'year', 'region', 'ladder score', 'upperwhisker', 'lowerwhisker', 'log gdp per capita', 'social support', 'healthy life expectancy', 'freedom to make life choices', 'generosity', 'perceptions of corruption', 'dystopia + residual']]
df2024.columns = [elem for elem in df2024.columns[:6]] + [f'{elem} factor' for elem in df2024.columns[6:]]

In [ ]:
df = df.sort_values(by=['country', 'year']).reset_index(drop=True)
df.head(5)

,country,year,region,life ladder,log gdp per capita,social support,healthy life expectancy at birth,freedom to make life choices,generosity,perceptions of corruption,positive affect,negative affect
0,Afghanistan,2008,South Asia,3.724,7.350,0.451,50.5,0.718,0.164,0.882,0.414,0.258
1,Afghanistan,2009,South Asia,4.402,7.509,0.552,50.8,0.679,0.187,0.850,0.481,0.237
2,Afghanistan,2010,South Asia,4.758,7.614,0.539,51.1,0.600,0.118,0.707,0.517,0.275
3,Afghanistan,2011,South Asia,3.832,7.581,0.521,51.4,0.496,0.160,0.731,0.480,0.267
4,Afghanistan,2012,South Asia,3.783,7.661,0.521,51.7,0.531,0.234,0.776,0.614,0.268


In [ ]:
df2024 = df2024.sort_values(by=['country', 'year']).reset_index(drop=True)
df2024.head(5)

,country,year,region,ladder score,upperwhisker,lowerwhisker,log gdp per capita factor,social support factor,healthy life expectancy factor,freedom to make life choices factor,generosity factor,perceptions of corruption factor,dystopia + residual factor
0,Afghanistan,2024,South Asia,1.721,1.775,1.667,0.628,0.000,0.242,0.000,0.091,0.088,0.672
1,Albania,2024,Central and Eastern Europe,5.304,5.440,5.168,1.438,0.924,0.638,0.690,0.138,0.049,1.428
2,Algeria,2024,Middle East and North Africa,5.364,5.476,5.251,1.324,1.191,0.568,0.247,0.091,0.200,1.743
3,Argentina,2024,Latin America and Caribbean,6.188,6.306,6.070,1.562,1.381,0.585,0.681,0.087,0.080,1.812
4,Armenia,2024,Commonwealth of Independent States,5.455,5.560,5.350,1.444,1.154,0.603,0.650,0.051,0.173,1.379


In [ ]:
df.to_csv(os.path.join(DATA_DIR, 'clean/world-happiness-2005-2023.csv'), index=False)
df2024.to_csv(os.path.join(DATA_DIR, 'clean/world-happiness-2024.csv'), index=False)

## Annual Temperature Data (2005-2023)

1. Adjusted column names for better handling
2. Removed data from irrelevant years (i.e., outside 2005-2023)
3. Adjusted country names to match those in World Happiness Data
4. Removed data of (countries, year) that are not listed in the world happiness data

In [ ]:
temp = pd.read_csv(os.path.join(DATA_DIR, "city-temperature.csv"), encoding='latin-1')
temp.head(5)

<ipython-input-487-c61aaf5b88cf>:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv(os.path.join(DATA_DIR, "city-temperature.csv"), encoding='latin-1')


,Region,Country,State,City,Month,Day,Year,AvgTemperature
0,Africa,Algeria,NaN,Algiers,1,1,1995,64.2
1,Africa,Algeria,NaN,Algiers,1,2,1995,49.4
2,Africa,Algeria,NaN,Algiers,1,3,1995,48.8
3,Africa,Algeria,NaN,Algiers,1,4,1995,46.4
4,Africa,Algeria,NaN,Algiers,1,5,1995,47.9


In [ ]:
columns = ['Region', 'State', 'City', 'Month', 'Day']
for col in columns:
    del temp[col]

temp = temp.groupby(['Country', 'Year'])['AvgTemperature'].mean().reset_index()
temp.head(5)

,Country,Year,AvgTemperature
0,Albania,1995,-99.000000
1,Albania,1996,-99.000000
2,Albania,1997,-88.228767
3,Albania,1998,-46.195890
4,Albania,1999,26.696164


In [ ]:
temp = temp[(temp['Year'] >= 2005) & (temp['Year'] <= 2023)].reset_index(drop=True)
temp.columns = [elem.lower().replace('avg', 'avg ') for elem in temp.columns]
temp.head(5)

,country,year,avg temperature
0,Albania,2005,59.935616
1,Albania,2006,58.569041
2,Albania,2007,60.890137
3,Albania,2008,59.790710
4,Albania,2009,59.401096


In [ ]:
temp_countries = set(temp['country'])
df_countries = set(df['country'])

print("Countries in the temperature dataset but not in the happiness dataset:")
for elem in (temp_countries - df_countries):
    print(f' - {elem}')
print("\nCountries in the happiness dataset but not in the temperature dataset:")
for elem in (df_countries - temp_countries):
    print(f' - {elem}')

Countries in the temperature dataset but not in the happiness dataset:
 - Turkey
 - North Korea
 - Barbados
 - Yugoslavia
 - Myanmar (Burma)
 - Macedonia
 - Guinea-Bissau
 - Czech Republic
 - Bahamas
 - US
 - Congo
 - Bermuda
 - The Netherlands
 - Equador

Countries in the happiness dataset but not in the temperature dataset:
 - Bosnia and Herzegovina
 - Congo (Kinshasa)
 - Mali
 - Niger
 - Cameroon
 - Kosovo
 - Maldives
 - Myanmar
 - Moldova
 - Comoros
 - Ghana
 - Czechia
 - Netherlands
 - Trinidad and Tobago
 - Slovenia
 - Somaliland region
 - Rwanda
 - Chile
 - Angola
 - Iran
 - North Macedonia
 - Luxembourg
 - Mauritius
 - Botswana
 - Cambodia
 - Libya
 - Armenia
 - United States
 - Zimbabwe
 - Liberia
 - Afghanistan
 - Lithuania
 - El Salvador
 - Estonia
 - Iraq
 - South Sudan
 - Azerbaijan
 - Bhutan
 - Eswatini
 - Congo (Brazzaville)
 - Serbia
 - Jamaica
 - Ecuador
 - Montenegro
 - Somalia
 - Chad
 - Paraguay
 - Malta
 - Yemen
 - Lesotho
 - Sudan
 - State of Palestine
 - Djibouti

In [ ]:
# Congo is divided into 2 and Serbia-Montenegro is divided into 2 in the happiness dataset
congo_rows = temp[temp['country'] == 'Congo']
congo = congo_rows.copy()
congo['country'] = 'Congo (Kinshasa)'

serbia_rows = temp[temp['country'] == 'Serbia-Montenegro']
serbia = serbia_rows.copy()
serbia['country'] = 'Montenegro'

temp = pd.concat([temp, congo, serbia], ignore_index=True)

In [ ]:
temp_countries_dict = {#'Bahamas': ,
                       #'Barbados': ,
                       #'Bermuda': ,
                       'Congo': 'Congo (Brazzaville)',
                       'Czech Republic': 'Czechia',
                       'Equador': 'Ecuador',
                       #'Guinea-Bissau': ,
                       #'Hong Kong': 'Hong Kong S.A.R. of China',
                       'Macedonia': 'North Macedonia',
                       'Myanmar (Burma)': 'Myanmar',
                       #'North Korea': ,
                       'Serbia-Montenegro': 'Serbia',
                       #'Taiwan': 'Taiwan Province of China',
                       'The Netherlands': 'Netherlands',
                       'Turkey': 'Türkiye',
                       'US': 'United States',
                       #'Yugoslavia':
                       }

In [ ]:
countries = temp['country']
countries_rev = []
for country in countries:
    try:
        countries_rev.append(temp_countries_dict[country])
    except:
        countries_rev.append(country)

temp['country'] = countries_rev

In [ ]:
df_entries = [f'{foo}-{boo}' for foo, boo in zip(df['country'], df['year'])]
temp['check'] = [f'{foo}-{boo}' for foo, boo in zip(temp['country'], temp['year'])]
temp = temp[temp['check'].isin(df_entries)]
del temp['check']
temp = temp.sort_values(by=['country', 'year']).reset_index(drop=True)
temp.head(10)

,country,year,avg temperature
0,Albania,2007,60.890137
1,Albania,2009,59.401096
2,Albania,2010,60.568767
3,Albania,2011,59.918630
4,Albania,2012,59.884426
5,Albania,2013,60.413151
6,Albania,2014,60.215616
7,Albania,2015,60.090164
8,Albania,2016,58.986885
9,Albania,2017,60.384658


In [ ]:
temp_countries = set(temp['country'])
df_countries = set(df['country'])

print("Countries in the temperature dataset but not in the happiness dataset:")
for elem in (temp_countries - df_countries):
    print(f' - {elem}')
print("\nCountries in the happiness dataset but not in the temperature dataset:")
for elem in (df_countries - temp_countries):
    print(f' - {elem}')

Countries in the temperature dataset but not in the happiness dataset:

Countries in the happiness dataset but not in the temperature dataset:
 - Bosnia and Herzegovina
 - Mali
 - Niger
 - Cameroon
 - Kosovo
 - Maldives
 - Moldova
 - Comoros
 - Ghana
 - Trinidad and Tobago
 - Slovenia
 - Somaliland region
 - Rwanda
 - Chile
 - Angola
 - Iran
 - Luxembourg
 - Mauritius
 - Botswana
 - Cambodia
 - Libya
 - Armenia
 - Burundi
 - Zimbabwe
 - Liberia
 - Afghanistan
 - Lithuania
 - El Salvador
 - Estonia
 - Iraq
 - South Sudan
 - Azerbaijan
 - Bhutan
 - Eswatini
 - Serbia
 - Jamaica
 - Montenegro
 - Somalia
 - Chad
 - Paraguay
 - Malta
 - Yemen
 - Lesotho
 - Sudan
 - State of Palestine
 - Djibouti
 - Burkina Faso
 - Guyana


In [ ]:
temp.to_csv(os.path.join(DATA_DIR, 'clean/temperature-2005-2023.csv'), index=False)

In [ ]:
print(f"Number of entries in world happiness dataset: {len(df)}")

Number of entries in world happiness dataset: 2363


In [ ]:
df = pd.merge(df, temp, on=['country', 'year'], how='outer')
df = df.sort_values(by=['country', 'year']).reset_index(drop=True)
df.head(5)

,country,year,region,life ladder,log gdp per capita,social support,healthy life expectancy at birth,freedom to make life choices,generosity,perceptions of corruption,positive affect,negative affect,avg temperature
0,Afghanistan,2008,South Asia,3.724,7.350,0.451,50.5,0.718,0.164,0.882,0.414,0.258,NaN
1,Afghanistan,2009,South Asia,4.402,7.509,0.552,50.8,0.679,0.187,0.850,0.481,0.237,NaN
2,Afghanistan,2010,South Asia,4.758,7.614,0.539,51.1,0.600,0.118,0.707,0.517,0.275,NaN
3,Afghanistan,2011,South Asia,3.832,7.581,0.521,51.4,0.496,0.160,0.731,0.480,0.267,NaN
4,Afghanistan,2012,South Asia,3.783,7.661,0.521,51.7,0.531,0.234,0.776,0.614,0.268,NaN


In [ ]:
print(f"Number of entries in world happiness dataset after outer merge: {len(df)}")

Number of entries in world happiness dataset after outer merge: 2363


## World Bank Data (2005-2023) -- Auxiliary

0. Reformat the stupid table...
1. Adjusted column names for better handling
2. Removed data from irrelevant years (i.e., outside 2005-2023)
3. Adjusted country names to match those in World Happiness Data
4. Removed data of (countries, year) that are not listed in the world happiness data

In [ ]:
wd = pd.read_csv(os.path.join(DATA_DIR, "world-bank-group.csv"), encoding='latin-1')
wd.head(5)

,Series Name,Series Code,Country Name,Country Code,2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007],2008 [YR2008],2009 [YR2009],...,2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022],2023 [YR2023]
0,Average precipitation in depth (mm per year),AG.LND.PRCP.MM,Afghanistan,AFG,327,327,327,327,327,327,...,327,327,327,327,327,327,327,..,..,..
1,Average precipitation in depth (mm per year),AG.LND.PRCP.MM,Albania,ALB,1485,1485,1485,1485,1485,1485,...,1485,1485,1485,1485,1485,1485,1485,..,..,..
2,Average precipitation in depth (mm per year),AG.LND.PRCP.MM,Algeria,DZA,89,89,89,89,89,89,...,89,89,89,89,89,89,89,..,..,..
3,Average precipitation in depth (mm per year),AG.LND.PRCP.MM,American Samoa,ASM,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
4,Average precipitation in depth (mm per year),AG.LND.PRCP.MM,Andorra,AND,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..


In [ ]:
del wd['Country Code']
del wd['Series Code']
del wd['2004 [YR2004]']
wd_cols = wd.columns
wd.columns = [('Y'+elem[:4]).replace('YSeri', 'element').replace('YCoun', 'country') for elem in wd_cols]

In [ ]:
def convert(elem):
    if elem == '..':
        return np.nan
    else:
        return float(elem)

for col in wd.columns[2:]:
    wd[col] = [convert(elem) for elem in wd[col]]
wd.head(5)

,element,country,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,...,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020,Y2021,Y2022,Y2023
0,Average precipitation in depth (mm per year),Afghanistan,327.0,327.0,327.0,327.0,327.0,327.0,327.0,327.0,...,327.0,327.0,327.0,327.0,327.0,327.0,327.0,NaN,NaN,NaN
1,Average precipitation in depth (mm per year),Albania,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,...,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,NaN,NaN,NaN
2,Average precipitation in depth (mm per year),Algeria,89.0,89.0,89.0,89.0,89.0,89.0,89.0,89.0,...,89.0,89.0,89.0,89.0,89.0,89.0,89.0,NaN,NaN,NaN
3,Average precipitation in depth (mm per year),American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Average precipitation in depth (mm per year),Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
columns = list(set(wd['element']))
columns.sort()
columns

['Access to clean fuels and technologies for cooking (% of population)',
 'Access to electricity (% of population)',
 'Agricultural land (% of land area)',
 'Agricultural methane emissions (% of total)',
 'Agricultural nitrous oxide emissions (% of total)',
 'Agricultural raw materials exports (% of merchandise exports)',
 'Agricultural raw materials imports (% of merchandise imports)',
 'Agriculture, forestry, and fishing, value added (% of GDP)',
 'Alternative and nuclear energy (% of total energy use)',
 'Annual freshwater withdrawals, total (% of internal resources)',
 'Arable land (% of land area)',
 'Armed forces personnel (% of total labor force)',
 'Arms exports (SIPRI trend indicator values)',
 'Arms imports (SIPRI trend indicator values)',
 'Automated teller machines (ATMs) (per 100,000 adults)',
 'Average precipitation in depth (mm per year)',
 'Average time to clear exports through customs (days)',
 'Average transaction cost of sending remittances from a specific country (%

In [ ]:
columns = ['country', 'year'] + [elem.lower() for elem in columns]
da = pd.DataFrame(columns=columns)
da.head()

,country,year,access to clean fuels and technologies for cooking (% of population),access to electricity (% of population),agricultural land (% of land area),agricultural methane emissions (% of total),agricultural nitrous oxide emissions (% of total),agricultural raw materials exports (% of merchandise exports),agricultural raw materials imports (% of merchandise imports),"agriculture, forestry, and fishing, value added (% of gdp)",...,"unemployment, total (% of total labor force) (modeled ilo estimate)","unemployment, youth total (% of total labor force ages 15-24) (modeled ilo estimate)",unmet need for contraception (% of married women ages 15-49),urban land area (sq. km),urban population (% of total population),urban population growth (annual %),voice and accountability: estimate,"vulnerable employment, total (% of total employment) (modeled ilo estimate)","wage and salaried workers, total (% of total employment) (modeled ilo estimate)",young people (ages 15-24) newly infected with hiv


In [ ]:
wd.head(5)

,element,country,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,...,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020,Y2021,Y2022,Y2023
0,Average precipitation in depth (mm per year),Afghanistan,327.0,327.0,327.0,327.0,327.0,327.0,327.0,327.0,...,327.0,327.0,327.0,327.0,327.0,327.0,327.0,NaN,NaN,NaN
1,Average precipitation in depth (mm per year),Albania,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,...,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,NaN,NaN,NaN
2,Average precipitation in depth (mm per year),Algeria,89.0,89.0,89.0,89.0,89.0,89.0,89.0,89.0,...,89.0,89.0,89.0,89.0,89.0,89.0,89.0,NaN,NaN,NaN
3,Average precipitation in depth (mm per year),American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Average precipitation in depth (mm per year),Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
class Entry:
    def __init__(self, country, year):
        self.country = country
        self.year = year
        self.elems = {'country': country,
                      'year': year}

    def update(self, key, value):
        self.elems[key.lower()] = float(value)

    def print(self):
        print(f'Entry for {self.country.upper()} in {self.year}...')
        for key, value in self.elems.items():
            print(f' - {key}: {value}')
        if len(self.elems.items()) == 0:
            print("EMPTY!")

In [ ]:
entries = dict()
countries = list(set(wd['country']))
years = range(2005, 2024)
for country in countries:
    for year in years:
        entries[f'{country}-{year}'] = Entry(country, year)

print(f"Number of entries in wd: {len(wd)}")

Number of entries in wd: 62244


In [ ]:
for row in tqdm(wd.itertuples(index=False)):
    element = row.element
    country = row.country
    entries[f'{country}-2005'].update(element, row.Y2005)
    entries[f'{country}-2006'].update(element, row.Y2006)
    entries[f'{country}-2007'].update(element, row.Y2007)
    entries[f'{country}-2008'].update(element, row.Y2008)
    entries[f'{country}-2009'].update(element, row.Y2009)
    entries[f'{country}-2010'].update(element, row.Y2010)
    entries[f'{country}-2011'].update(element, row.Y2011)
    entries[f'{country}-2012'].update(element, row.Y2012)
    entries[f'{country}-2013'].update(element, row.Y2013)
    entries[f'{country}-2014'].update(element, row.Y2014)
    entries[f'{country}-2015'].update(element, row.Y2015)
    entries[f'{country}-2016'].update(element, row.Y2016)
    entries[f'{country}-2017'].update(element, row.Y2017)
    entries[f'{country}-2018'].update(element, row.Y2018)
    entries[f'{country}-2019'].update(element, row.Y2019)
    entries[f'{country}-2020'].update(element, row.Y2020)
    entries[f'{country}-2021'].update(element, row.Y2021)
    entries[f'{country}-2022'].update(element, row.Y2022)
    entries[f'{country}-2023'].update(element, row.Y2023)

62244it [00:01, 34146.18it/s]


In [ ]:
entries['Afghanistan-2005'].print()

Entry for AFGHANISTAN in 2005...
 - country: Afghanistan
 - year: 2005
 - average precipitation in depth (mm per year): 327.0
 - access to clean fuels and technologies for cooking (% of population): 12.5
 - access to electricity (% of population): 28.66967201
 - agricultural land (% of land area): 58.13440044
 - agricultural methane emissions (% of total): 72.83633248
 - agricultural nitrous oxide emissions (% of total): 90.17632242
 - agricultural raw materials exports (% of merchandise exports): nan
 - agricultural raw materials imports (% of merchandise imports): nan
 - agriculture, forestry, and fishing, value added (% of gdp): 31.11485491
 - alternative and nuclear energy (% of total energy use): nan
 - annual freshwater withdrawals, total (% of internal resources): 43.01590668
 - arable land (% of land area): 11.96663754
 - automated teller machines (atms) (per 100,000 adults): 0.06
 - arms exports (sipri trend indicator values): nan
 - arms imports (sipri trend indicator values)

In [ ]:
entries_list = []
for key, value in entries.items():
    entries_list.append(value.elems)

In [ ]:
for entry in tqdm(entries_list):
    entry_df = pd.DataFrame([entry])
    da = pd.concat([da, entry_df], ignore_index=True)

100%|██████████| 5054/5054 [03:19<00:00, 25.36it/s]


In [ ]:
da = da.sort_values(by=['country', 'year']).reset_index(drop=True)
da.head(10)

,country,year,access to clean fuels and technologies for cooking (% of population),access to electricity (% of population),agricultural land (% of land area),agricultural methane emissions (% of total),agricultural nitrous oxide emissions (% of total),agricultural raw materials exports (% of merchandise exports),agricultural raw materials imports (% of merchandise imports),"agriculture, forestry, and fishing, value added (% of gdp)",...,"unemployment, total (% of total labor force) (modeled ilo estimate)","unemployment, youth total (% of total labor force ages 15-24) (modeled ilo estimate)",unmet need for contraception (% of married women ages 15-49),urban land area (sq. km),urban population (% of total population),urban population growth (annual %),voice and accountability: estimate,"vulnerable employment, total (% of total employment) (modeled ilo estimate)","wage and salaried workers, total (% of total employment) (modeled ilo estimate)",young people (ages 15-24) newly infected with hiv
0,Afghanistan,2005,12.5,28.669672,58.134400,72.836332,90.176322,NaN,NaN,31.114855,...,7.925,10.176,NaN,NaN,22.703,4.474690,-1.125430,91.256812,8.336504,500.0
1,Afghanistan,2006,13.9,33.544418,58.123668,58.092680,83.743842,NaN,NaN,28.635969,...,7.925,10.177,NaN,NaN,22.907,5.034216,-1.110294,91.072010,8.503913,500.0
2,Afghanistan,2007,15.3,38.440002,58.129801,48.618785,78.117647,NaN,NaN,30.105011,...,7.924,10.172,NaN,NaN,23.113,2.688468,-1.057654,90.813159,8.727813,500.0
3,Afghanistan,2008,16.8,42.400000,58.132867,44.203879,75.156576,9.894829,1.476686,24.892270,...,7.928,10.165,NaN,NaN,23.320,2.893949,-1.168902,90.607746,8.903806,500.0
4,Afghanistan,2009,18.2,48.279007,58.132867,NaN,NaN,7.559673,NaN,29.297501,...,7.923,10.148,NaN,NaN,23.528,4.449269,-1.375495,89.984422,9.409127,500.0
5,Afghanistan,2010,19.7,42.700000,58.134400,NaN,NaN,10.767046,NaN,26.210069,...,7.921,10.133,NaN,NaN,23.737,3.779279,-1.404467,89.316757,9.950540,500.0
6,Afghanistan,2011,21.3,43.222019,58.131334,NaN,NaN,12.879009,NaN,23.743664,...,7.918,10.115,NaN,NaN,23.948,4.574493,-1.335977,88.688147,10.459113,500.0
7,Afghanistan,2012,22.7,69.100000,58.129801,NaN,NaN,0.236604,NaN,24.390874,...,7.914,10.098,NaN,NaN,24.160,4.958981,-1.267430,87.943290,11.037459,500.0
8,Afghanistan,2013,24.3,68.040878,58.123668,NaN,NaN,0.036992,NaN,22.810663,...,7.914,10.092,NaN,NaN,24.373,4.344553,-1.240068,86.864769,11.574016,500.0
9,Afghanistan,2014,25.7,89.500000,58.123668,NaN,NaN,0.047523,NaN,22.137041,...,7.910,10.086,NaN,NaN,24.587,4.531769,-1.135440,85.615463,12.037376,500.0


In [ ]:
len(set(da['country'])), len(set(df['country']))

(266, 165)

In [ ]:
da_countries = set(da['country'])
df_countries = set(df['country'])

print("Countries in the world bank dataset but not in the happiness dataset:")
for elem in (da_countries - df_countries):
    print(f" - {elem}")
print("\nCountries in the happiness dataset but not in the world bank dataset:")
for elem in (df_countries - da_countries):
    print(f" - {elem}")

Countries in the world bank dataset but not in the happiness dataset:
 - Heavily indebted poor countries (HIPC)
 - Cabo Verde
 - Pacific island small states
 - Sub-Saharan Africa (IDA & IBRD countries)
 - Slovak Republic
 - Middle East & North Africa (excluding high income)
 - Palau
 - Least developed countries: UN classification
 - Micronesia, Fed. Sts.
 - Congo, Rep.
 - Nauru
 - Europe & Central Asia (excluding high income)
 - Tuvalu
 - South Asia (IDA & IBRD)
 - Iran, Islamic Rep.
 - Aruba
 - Sub-Saharan Africa (excluding high income)
 - Latin America & the Caribbean (IDA & IBRD countries)
 - Korea, Dem. People's Rep.
 - Fiji
 - St. Martin (French part)
 - OECD members
 - Euro area
 - Middle East & North Africa
 - Barbados
 - Brunei Darussalam
 - Caribbean small states
 - Syrian Arab Republic
 - East Asia & Pacific (IDA & IBRD countries)
 - Sub-Saharan Africa
 - Fragile and conflict affected situations
 - Africa Western and Central
 - Low & middle income
 - West Bank and Gaza
 - Ant

In [ ]:
df_countries - da_countries

{'Congo (Brazzaville)',
 'Congo (Kinshasa)',
 'Egypt',
 'Gambia',
 'Hong Kong',
 'Iran',
 'Ivory Coast',
 'Kyrgyzstan',
 'Laos',
 'Russia',
 'Slovakia',
 'Somaliland region',
 'South Korea',
 'State of Palestine',
 'Syria',
 'Taiwan',
 'Türkiye',
 'Venezuela',
 'Vietnam',
 'Yemen'}

In [ ]:
da_countries_dict = {
    #"Heavily indebted poor countries (HIPC)": "",
    #"Cabo Verde": "",
    #"Pacific island small states": "",
    #"Sub-Saharan Africa (IDA & IBRD countries)": "",
    "Slovak Republic": "Slovakia",
    #"Middle East & North Africa (excluding high income)": "",
    #"Palau": "",
    #"Least developed countries: UN classification": "",
    #"Micronesia, Fed. Sts.": "",
    "Congo, Rep.": "Congo (Brazzaville)",
    #"Nauru": "",
    #"Europe & Central Asia (excluding high income)": "",
    #"Tuvalu": "",
    #"South Asia (IDA & IBRD)": "",
    "Iran, Islamic Rep.": "Iran",
    #"Aruba": "",
    #"Sub-Saharan Africa (excluding high income)": "",
    #"Latin America & the Caribbean (IDA & IBRD countries)": "",
    #"Korea, Dem. People's Rep.": "",
    #"Fiji": "",
    #"St. Martin (French part)": "",
    #"OECD members": "",
    #"Euro area": "",
    #"Middle East & North Africa": "",
    #"Barbados": "",
    #"Brunei Darussalam": "",
    #"Caribbean small states": "",
    "Syrian Arab Republic": "Syria",
    #"East Asia & Pacific (IDA & IBRD countries)": "",
    #"Sub-Saharan Africa": "",
    #"Fragile and conflict affected situations": "",
    #"Africa Western and Central": "",
    #"Low & middle income": "",
    #"West Bank and Gaza": "",
    #"Antigua and Barbuda": "",
    #"Channel Islands": "",
    #"Sint Maarten (Dutch part)": "",
    #"North America": "",
    #"Sao Tome and Principe": "",
    #"Latin America & Caribbean": "",
    #"Guam": "",
    "Korea, Rep.": "South Korea",
    #"Europe & Central Asia": "",
    #"World": "",
    #"Eritrea": "",
    #"IDA blend": "",
    "Lao PDR": "Laos",
    #"Liechtenstein": "",
    #"Greenland": "",
    #"Macao SAR, China": "",
    "Viet Nam": "Vietnam",
    #"IDA & IBRD total": "",
    #"San Marino": "",
    "Congo, Dem. Rep.": "Congo (Kinshasa)",
    #"British Virgin Islands": "",
    #"Samoa": "",
    #"Dominica": "",
    "Gambia, The": "Gambia",
    #"Monaco": "",
    #"Arab World": "",
    #"Kiribati": "",
    #"Upper middle income": "",
    #"Guinea-Bissau": "",
    "Egypt, Arab Rep.": "Egypt",
    #"Cayman Islands": "",
    #"Late-demographic dividend": "",
    #"IDA only": "",
    #"Equatorial Guinea": "",
    #"South Asia": "",
    #"St. Vincent and the Grenadines": "",
    #"Other small states": "",
    #"Isle of Man": "",
    #"Papua New Guinea": "",
    #"St. Lucia": "",
    #"Marshall Islands": "",
    #"IBRD only": "",
    #"Northern Mariana Islands": "",
    #"Early-demographic dividend": "",
    #"European Union": "",
    #"New Caledonia": "",
    #"Puerto Rico": "",
    #"American Samoa": "",
    #"Middle East & North Africa (IDA & IBRD countries)": "",
    #"Bermuda": "",
    #"Pre-demographic dividend": "",
    "Yemen, Rep.": "Yemen",
    #"Seychelles": "",
    "Kyrgyz Republic": "Kyrgyzstan",
    #"Faroe Islands": "",
    #"IDA total": "",
    #"Small states": "",
    #"Turks and Caicos Islands": "",
    #"French Polynesia": "",
    #"Grenada": "",
    #"East Asia & Pacific (excluding high income)": "",
    #"Europe & Central Asia (IDA & IBRD countries)": "",
    #"Africa Eastern and Southern": "",
    "Russian Federation": "Russia",
    #"Solomon Islands": "",
    #"Central Europe and the Baltics": "",
    #"Gibraltar": "",
    #"Low income": "",
    #"St. Kitts and Nevis": "",
    #"Middle income": "",
    #"Vanuatu": "",
    #"Lower middle income": "",
    #"Not classified": "",
    #"Timor-Leste": "",
    "Venezuela, RB": "Venezuela",
    #"Andorra": "",
    #"High income": "",
    #"Tonga": "",
    #"Post-demographic dividend": "",
    #"East Asia & Pacific": "",
    "Turkiye": "Türkiye",
    #"Bahamas, The": "",
    #"Virgin Islands (U.S.)": "",
    #"Latin America & Caribbean (excluding high income)": "",
    "Hong Kong SAR, China": "Hong Kong",
    #"Cote d'Ivoire": "",
    #"Curacao": ""
}

In [ ]:
countries = da['country']
countries_rev = []
for country in countries:
    try:
        countries_rev.append(da_countries_dict[country])
    except:
        countries_rev.append(country)

da['country'] = countries_rev

In [ ]:
df_entries = [f'{foo}-{boo}' for foo, boo in zip(df['country'], df['year'])]
da['check'] = [f'{foo}-{boo}' for foo, boo in zip(da['country'], da['year'])]
da = da[da['check'].isin(df_entries)]
del da['check']
da = da.sort_values(by=['country', 'year']).reset_index(drop=True)
da.head(10)

,country,year,access to clean fuels and technologies for cooking (% of population),access to electricity (% of population),agricultural land (% of land area),agricultural methane emissions (% of total),agricultural nitrous oxide emissions (% of total),agricultural raw materials exports (% of merchandise exports),agricultural raw materials imports (% of merchandise imports),"agriculture, forestry, and fishing, value added (% of gdp)",...,"unemployment, total (% of total labor force) (modeled ilo estimate)","unemployment, youth total (% of total labor force ages 15-24) (modeled ilo estimate)",unmet need for contraception (% of married women ages 15-49),urban land area (sq. km),urban population (% of total population),urban population growth (annual %),voice and accountability: estimate,"vulnerable employment, total (% of total employment) (modeled ilo estimate)","wage and salaried workers, total (% of total employment) (modeled ilo estimate)",young people (ages 15-24) newly infected with hiv
0,Afghanistan,2008,16.80,42.400000,58.132867,44.203879,75.156576,9.894829,1.476686,24.892270,...,7.928,10.165,NaN,NaN,23.320,2.893949,-1.168902,90.607746,8.903806,500.0
1,Afghanistan,2009,18.20,48.279007,58.132867,NaN,NaN,7.559673,NaN,29.297501,...,7.923,10.148,NaN,NaN,23.528,4.449269,-1.375495,89.984422,9.409127,500.0
2,Afghanistan,2010,19.70,42.700000,58.134400,NaN,NaN,10.767046,NaN,26.210069,...,7.921,10.133,NaN,NaN,23.737,3.779279,-1.404467,89.316757,9.950540,500.0
3,Afghanistan,2011,21.30,43.222019,58.131334,NaN,NaN,12.879009,NaN,23.743664,...,7.918,10.115,NaN,NaN,23.948,4.574493,-1.335977,88.688147,10.459113,500.0
4,Afghanistan,2012,22.70,69.100000,58.129801,NaN,NaN,0.236604,NaN,24.390874,...,7.914,10.098,NaN,NaN,24.160,4.958981,-1.267430,87.943290,11.037459,500.0
5,Afghanistan,2013,24.30,68.040878,58.123668,NaN,NaN,0.036992,NaN,22.810663,...,7.914,10.092,NaN,NaN,24.373,4.344553,-1.240068,86.864769,11.574016,500.0
6,Afghanistan,2014,25.70,89.500000,58.123668,NaN,NaN,0.047523,NaN,22.137041,...,7.910,10.086,NaN,NaN,24.587,4.531769,-1.135440,85.615463,12.037376,500.0
7,Afghanistan,2015,27.25,71.500000,58.123668,NaN,NaN,14.799067,NaN,20.634323,...,9.002,12.507,24.500000,3993.836107,24.803,3.996008,-1.117557,85.139009,12.612444,500.0
8,Afghanistan,2016,28.50,97.700000,58.123668,NaN,NaN,16.359109,NaN,25.740314,...,10.092,14.909,24.465346,NaN,25.020,3.452643,-1.037788,84.636752,13.217040,500.0
9,Afghanistan,2017,30.00,97.700000,58.123668,NaN,NaN,17.063634,1.871025,26.420199,...,11.180,17.303,NaN,NaN,25.250,3.781557,-0.990814,84.029744,13.904849,500.0


In [ ]:
da_countries = set(da['country'])
df_countries = set(df['country'])

print("Countries in the world bank dataset but not in the happiness dataset:")
for elem in (da_countries - df_countries):
    print(f" - {elem}")
print("\nCountries in the happiness dataset but not in the world bank dataset:")
for elem in (df_countries - da_countries):
    print(f" - {elem}")

Countries in the world bank dataset but not in the happiness dataset:

Countries in the happiness dataset but not in the world bank dataset:
 - Taiwan
 - State of Palestine
 - Somaliland region
 - Ivory Coast


In [ ]:
da.to_csv(os.path.join(DATA_DIR, 'clean/world-bank-data-2005-2023.csv'), index=False)

In [ ]:
print(f"Number of entries in world happiness dataset: {len(df)}")

Number of entries in world happiness dataset: 2363


In [ ]:
df = pd.merge(df, da, on=['country', 'year'], how='outer')
df = df.sort_values(by=['country', 'year']).reset_index(drop=True)
df.head(5)

,country,year,region,life ladder,log gdp per capita,social support,healthy life expectancy at birth,freedom to make life choices,generosity,perceptions of corruption,...,"unemployment, total (% of total labor force) (modeled ilo estimate)","unemployment, youth total (% of total labor force ages 15-24) (modeled ilo estimate)",unmet need for contraception (% of married women ages 15-49),urban land area (sq. km),urban population (% of total population),urban population growth (annual %),voice and accountability: estimate,"vulnerable employment, total (% of total employment) (modeled ilo estimate)","wage and salaried workers, total (% of total employment) (modeled ilo estimate)",young people (ages 15-24) newly infected with hiv
0,Afghanistan,2008,South Asia,3.724,7.350,0.451,50.5,0.718,0.164,0.882,...,7.928,10.165,NaN,NaN,23.320,2.893949,-1.168902,90.607746,8.903806,500.0
1,Afghanistan,2009,South Asia,4.402,7.509,0.552,50.8,0.679,0.187,0.850,...,7.923,10.148,NaN,NaN,23.528,4.449269,-1.375495,89.984422,9.409127,500.0
2,Afghanistan,2010,South Asia,4.758,7.614,0.539,51.1,0.600,0.118,0.707,...,7.921,10.133,NaN,NaN,23.737,3.779279,-1.404467,89.316757,9.950540,500.0
3,Afghanistan,2011,South Asia,3.832,7.581,0.521,51.4,0.496,0.160,0.731,...,7.918,10.115,NaN,NaN,23.948,4.574493,-1.335977,88.688147,10.459113,500.0
4,Afghanistan,2012,South Asia,3.783,7.661,0.521,51.7,0.531,0.234,0.776,...,7.914,10.098,NaN,NaN,24.160,4.958981,-1.267430,87.943290,11.037459,500.0


In [ ]:
print(f"Number of entries in world happiness dataset after outer merge: {len(df)}")

Number of entries in world happiness dataset after outer merge: 2363


In [ ]:
df.to_csv(os.path.join(DATA_DIR, 'clean/combined.csv'), index=False)